In [36]:
from pprint import pprint

import pandas as pd
import numpy as np
import re

%matplotlib inline
import matplotlib.pyplot as plt

In [34]:
#This will get our TF-IDF score without having to find out TF or IDF on our own
from sklearn.feature_extraction.text import TfidfVectorizer
#For making the Test-Train split to measure how well the model performed
from sklearn.model_selection import train_test_split
#We're going to use the Decision Tree alogrithm to classify the content 
from sklearn.tree import DecisionTreeClassifier
#We will use these tools to measure how well we classified the categories based on the content
from sklearn.metrics import classification_report, accuracy_score

In [2]:
#We are going basic clean, like almost always
#Went with lemmatize. Don't have the a good reason to say why.
from prepare import basic_clean, lemmatize

#That's the function that gets articles from InSHort.com. We've already got a csv it's pulling from.
from acquire import get_articles

In [35]:
#This is in acquire now. There were a lot of numbers in the words/features.
def remove_numbers(text):
    text = re.sub(r"[0-9]", '', text)
    return text

In [6]:
df = get_articles()
#The index column was annoying.
df.set_index('Unnamed: 0',inplace=True)

SAMPLE BEFORE USING THE ENTIRE CORPUS OF DOCUMENTS

In [13]:
document = df.content[0]

document

'after a u jury found that elon musk did not defame british cave explorer vernon unsworth by calling him a pedo guy on twitter the tesla ceo said my faith in humanity is restored the 48yearold billionaire argued that he did not intend to call unsworth a paedophile but instead wa using pedo guy to mean creepy old guy'

In [14]:
document = basic_clean(document)

document = lemmatize(document)

words = pd.Series(document.split())

In [21]:
#The tf-idf sklearn object that will derive the tf-idf score of each word/feature. 
tfidf = TfidfVectorizer() 

In [40]:
#A little sampling to see how these modules and functions work.
just_content = df.content

just_content = just_content.apply(basic_clean)

just_content = just_content.apply(lemmatize)

just_content = just_content.apply(remove_numbers)

tfidfs = tfidf.fit_transform(just_content.values)

In [41]:
#This will be a DataFrame with each word/feature as a column.
#It'll be mostly zeros, but the ones with any sort of number have a tf-idf above zero.
#Meaning they are frequent within the document(row#) yet uncommon among the corpus(all rows)
pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())

,aaryan,ab,abbey,ability,able,about,abstoxic,abuse,accent,accept,...,young,younger,your,yourself,yourstory,zero,zilingo,zima,zomato,zurich
0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
1,0.000000,0.000000,0.000000,0.0,0.000000,0.101990,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000
7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.15348
8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.142724,0.000000,0.0,0.0000,0.0,0.00000
9,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.00000


In [59]:
#I thought we had 100 rows? 20 a piece. Oh Well.
df.shape

(99, 5)

In [43]:
#Preparing the content column with our Prepare.py functions
df.content = df.content.apply(remove_numbers)
df.content = df.content.apply(basic_clean)
df.content = df.content.apply(lemmatize)

In [45]:
#X is is the Feature Matrix of TF-IDF scores per word.
X = tfidf.fit_transform(df.content)
#Target variable is the category. We want to predict each article's category by using the words/features in its content
y = df.category

In [48]:
#Split them up using the sklean-testTrain split funciton
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

In [60]:
#TWe will make fresh DataFrames for the train and test. To start we will give them the actual category values as the
#only column
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

In [69]:
#STEP 1-2 of the SK-LEARN API:
# 1_Call the object, 2_Fit the object to the training(feature matrix, target_variable)
tree = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)

In [71]:
# 3_Transform/Predict is assigned to a column in the Train and Test dataframes. 
#They should now be 2 columns wide. ['actual','predicted']
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

In [76]:
train

,actual,predicted
Unnamed: 0,,
51,technology,technology
91,entertainment,entertainment
12,business,technology
7,business,technology
61,technology,technology
4,business,technology
86,entertainment,entertainment
58,technology,technology
63,technology,technology


In [74]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))

Accuracy: 56.96%
---
Confusion Matrix
actual         business  entertainment  sports  technology
predicted                                                 
entertainment         0             15       2           0
sports                0              0      10           0
technology           19              5       8          20
---
               precision    recall  f1-score   support

     business       0.00      0.00      0.00        19
entertainment       0.88      0.75      0.81        20
       sports       1.00      0.50      0.67        20
   technology       0.38      1.00      0.56        20

     accuracy                           0.57        79
    macro avg       0.57      0.56      0.51        79
 weighted avg       0.57      0.57      0.51        79



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [75]:
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.predicted, test.actual))
print('---')
print(classification_report(test.actual, test.predicted))

Accuracy: 35.00%
---
Confusion Matrix
actual         business  entertainment  sports  technology
predicted                                                 
entertainment         1              1       0           0
sports                0              0       1           0
technology            4              4       4           5
---
               precision    recall  f1-score   support

     business       0.00      0.00      0.00         5
entertainment       0.50      0.20      0.29         5
       sports       1.00      0.20      0.33         5
   technology       0.29      1.00      0.45         5

     accuracy                           0.35        20
    macro avg       0.45      0.35      0.27        20
 weighted avg       0.45      0.35      0.27        20

